# Loading Data & Trained Model

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt

# 학습 데이터 불러오기
import pickle

# 학습 모델 불러오기
from tensorflow.keras.models import load_model

ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# 불용어
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()

[0.013s][warning][os,thread] Attempt to protect stack guard pages failed (0x000000016d3f8000-0x000000016d404000).
[0.013s][warning][os,thread] Attempt to deallocate stack guard pages failed.


In [3]:
# 전처리된 데이터 불러오기

with open("X_train.pkl","rb") as f:
    X_train = pickle.load(f)
    
with open("X_test.pkl","rb") as f:
    X_test = pickle.load(f)

y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

In [4]:
# 정수 인코딩 후 빈 샘플 제거
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)


# 빈 샘플들을 제거
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)

단어 집합(vocabulary)의 크기 : 43752
등장 빈도가 2번 이하인 희귀 단어의 수: 24337
단어 집합에서 희귀 단어의 비율: 55.62488571950996
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 1.8715872104872904
단어 집합의 크기 : 19416


/Users/kunwooshin/miniforge3/envs/tensorflow/lib/python3.8/site-packages/numpy/lib/function_base.py:5071: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [5]:
# 패딩

def below_threshold_len(max_len, nested_list):
    count = 0
    for sentence in nested_list:
        if(len(sentence) <= max_len):
            count = count + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

max_len = 30
below_threshold_len(max_len, X_train)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

전체 샘플 중 길이가 30 이하인 샘플의 비율: 94.31944999380003


In [6]:
from tensorflow.keras.models import load_model

In [7]:
# 학습된 모델을 불러옵니다.

loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

Metal device set to: Apple M1


2022-12-14 18:35:00.155973: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-14 18:35:00.156122: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-12-14 18:35:00.615503: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-14 18:35:00.795221: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


   1/1527 [..............................] - ETA: 10:02 - loss: 0.2928 - acc: 0.9062

2022-12-14 18:35:00.854618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1527/1527 [==============================] - 14s 9ms/step - loss: 0.3419 - acc: 0.8576

 테스트 정확도: 0.8576


# Sentiment Analysis

In [8]:
# 리뷰별 감성분석
def sentiment_predict(new_sentence):
    if new_sentence == 'nan':
        return float('Nan') , -1
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        return round(score*100,2), 1
    else:
        return round(-(1-score)*100,2), 0

In [9]:
# 긍부정점수+개수 변수 추가 (수화님 코드)
def make_pn(df): 
    df['review'] = df['review'].astype('str')
    df['review']=df['review'].apply(lambda x :x.replace('_x000D_','')
                                    .replace('관람객','').replace('스포일러가 포함된 감상평입니다. 감상평 보기',''))
    result=df['review'].apply(sentiment_predict)
    declist=[]
    numlist=[]

    for i in range(len(result)):
        dec = result[i][0]
        num = result[i][1]
        declist.append(dec)
        numlist.append(num)
    df['prob']=declist
    df['pos/neg']=numlist
    # 새로 저장할 디렉토리 설정
    return df.to_excel('/Users/kunwooshin/movie_scraping/review_file_rated/'+name+'_review.xlsx')

In [10]:
import os

# 스크랩 파일이 현재 있는 디렉토리 (각자 배분된 파일만 하시면 될 것 같아요)
dir_path = "/Users/kunwooshin/movie_scraping/review"

review_lst = []

for (root, directories, files) in os.walk(dir_path):
    for d in directories:
        d_path = os.path.join(root, d)
        print(d_path)

    for file in files:
        file_path = os.path.join(root, file)
        review_lst.append(file_path)

review_lst.remove('/Users/kunwooshin/movie_scraping/review/.DS_Store')

In [ ]:
review_lst

In [12]:
for elem in review_lst:
    df = pd.read_excel(elem, engine='openpyxl')
    # 감성분석된 파일 저장할 디렉토리 (각자 자유롭게 설정)
    name = elem.replace('/Users/kunwooshin/movie_scraping/review/','').replace('_review.xlsx','')
    make_pn(df)


1/1 [==============================] - 0s 276ms/step


2022-12-14 18:36:29.906639: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-14 18:36:29.944579: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 13ms/step


...

1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


1/1 [==============================] - 0s 14ms/step


# DataFrame 생성

In [51]:
# 전체 영화 별점, 긍부정점수 담을 데이터프레임 생성
total = pd.DataFrame()

In [39]:
import os

# 감성분석된 리뷰 스크랩 파일들이 있는 디렉토리
dir_path = "/Users/kunwooshin/movie_scraping/review_file_rated"

review_lst = []

for (root, directories, files) in os.walk(dir_path):
    for d in directories:
        d_path = os.path.join(root, d)
        print(d_path)

    for file in files:
        file_path = os.path.join(root, file)
        review_lst.append(file_path)
        
review_lst.remove('/Users/kunwooshin/movie_scraping/review_file_rated/.DS_Store')

In [46]:
# 188개가 나오는 게 정상!
review_lst

['/Users/kunwooshin/movie_scraping/review_file_rated/공조2_ 인터내셔날_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/더 배트맨_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/한산_ 용의 출현_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/외계+인 1부_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/쥬라기 월드_ 도미니언_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/헤어질 결심_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/헌트_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/미니언즈2_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/브로커_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/닥터 스트레인지_ 대혼돈의 멀티버스_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/신비한 동물들과 덤블도어의 비밀_review.xlsx',
 '/Users/kunwooshin/movie_scraping/review_file_rated/ᄇ

In [47]:
len(review_lst)

16

In [48]:
import pandas as pd
import numpy as np

In [49]:
score_lst = []
name_lst = []
pos_avg_lst = []
neg_avg_lst = []
std_lst = []
pos_ratio_lst = []

for elem in review_lst:
    df = pd.read_excel(elem, engine='openpyxl')
    
    # 별점 평균
    score = np.mean(df['score'])
    
    # 긍/부정 점수 각각의 평균
    pos = []
    neg = []
    for x in df['prob']:
        if x > 0:
            pos.append(x)
        elif x < 0:
            neg.append(x)
    pos_avg = sum(pos)/len(pos)
    neg_avg = sum(neg)/len(neg)
    
    # 총 긍/부정 점수 표준편차
    std = np.std(pos+neg)
    
    # 긍정 리뷰 비율
    pos_ratio = len(pos)/(len(pos)+len(neg))
    
    name = elem.replace('/Users/kunwooshin/movie_scraping/review_file_rated/','').replace('_review.xlsx','')
    
    score_lst.append(score)
    name_lst.append(name)
    pos_avg_lst.append(pos_avg)
    neg_avg_lst.append(neg_avg)
    std_lst.append(std)
    pos_ratio_lst.append(pos_ratio)

In [52]:
total['score'] = score_lst
total['name'] = name_lst
total['pos_avg'] = pos_avg_lst
total['neg_avg'] = neg_avg_lst
total['pos_ratio'] = pos_ratio_lst
total['std'] = std_lst

In [53]:
total.sort_values(by=['name'], inplace=True)

In [54]:
movie_info = pd.read_csv('test_dataset.csv')

In [55]:
movie_info.sort_values(by=['영화명'], inplace=True)

In [56]:
movie_info.head()

,Unnamed: 0,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국 스크린수,전국 매출액,전국 관객수,서울 매출액,서울 관객수,장르,등급,영화구분,개봉연도
3,4,공조2: 인터내셔날,이석훈,"(주)제이케이필름,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2022.9.7,개봉영화,장편,한국,"2,389","70,671,674,961","6,948,819","14,981,845,113","1,438,308",액션,15세이상관람가,일반영화,NaN
4,5,닥터 스트레인지: 대혼돈의 멀티버스,샘 레이미,NaN,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2022.5.4,개봉영화,장편,미국,"2,691","62,648,830,870","5,884,600","18,021,292,750","1,610,660",액션,12세이상관람가,일반영화,NaN
15,20,더 배트맨,맷 리브스,NaN,워너브러더스 코리아(주),워너브러더스 코리아(주),2022.3.1,개봉영화,장편,미국,"2,370","9,252,877,980","897,836","3,468,610,850","321,562",액션,15세이상관람가,일반영화,NaN
7,8,마녀(魔女) Part2. The Other One,박훈정,"(주)영화사 금월,(주)스튜디오앤뉴,(주)페퍼민트앤컴퍼니",NaN,(주)넥스트엔터테인먼트월드(NEW),2022.6.15,개봉영화,장편,한국,"1,796","28,920,577,670","2,806,277","6,020,469,150","570,915",액션,15세이상관람가,일반영화,NaN
9,10,미니언즈2,카일 발다,NaN,유니버설픽쳐스인터내셔널 코리아(유),유니버설픽쳐스인터내셔널 코리아(유),2022.7.20,개봉영화,장편,미국,"1,394","21,996,551,672","2,268,518","4,466,385,398","436,038",애니메이션,전체관람가,일반영화,NaN


In [57]:
movie_info.reset_index(inplace=True, drop=True)
total.reset_index(inplace=True, drop=True)


In [58]:
total.head()

,score,name,pos_avg,neg_avg,pos_ratio,std
0,8.791200,공조2_ 인터내셔날,93.297723,-85.397855,0.806760,71.549156
1,7.185400,닥터 스트레인지_ 대혼돈의 멀티버스,87.393728,-85.986724,0.525724,87.795379
2,7.165800,더 배트맨,87.518002,-87.410354,0.587533,87.248358
3,6.843200,마녀(魔女) Part2. The Other One,88.952339,-89.458399,0.512230,90.208853
4,7.681496,미니언즈2,91.135864,-85.340623,0.690639,82.625240


In [59]:
result = pd.concat([movie_info, total],axis=1)

In [60]:
result

,Unnamed: 0,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,...,장르,등급,영화구분,개봉연도,score,name,pos_avg,neg_avg,pos_ratio,std
0,4,공조2: 인터내셔날,이석훈,"(주)제이케이필름,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2022.9.7,개봉영화,장편,한국,...,액션,15세이상관람가,일반영화,NaN,8.791200,공조2_ 인터내셔날,93.297723,-85.397855,0.806760,71.549156
1,5,닥터 스트레인지: 대혼돈의 멀티버스,샘 레이미,NaN,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2022.5.4,개봉영화,장편,미국,...,액션,12세이상관람가,일반영화,NaN,7.185400,닥터 스트레인지_ 대혼돈의 멀티버스,87.393728,-85.986724,0.525724,87.795379
2,20,더 배트맨,맷 리브스,NaN,워너브러더스 코리아(주),워너브러더스 코리아(주),2022.3.1,개봉영화,장편,미국,...,액션,15세이상관람가,일반영화,NaN,7.165800,더 배트맨,87.518002,-87.410354,0.587533,87.248358
3,8,마녀(魔女) Part2. The Other One,박훈정,"(주)영화사 금월,(주)스튜디오앤뉴,(주)페퍼민트앤컴퍼니",NaN,(주)넥스트엔터테인먼트월드(NEW),2022.6.15,개봉영화,장편,한국,...,액션,15세이상관람가,일반영화,NaN,6.843200,마녀(魔女) Part2. The Other One,88.952339,-89.458399,0.512230,90.208853
4,10,미니언즈2,카일 발다,NaN,유니버설픽쳐스인터내셔널 코리아(유),유니버설픽쳐스인터내셔널 코리아(유),2022.7.20,개봉영화,장편,미국,...,애니메이션,전체관람가,일반영화,NaN,7.681496,미니언즈2,91.135864,-85.340623,0.690639,82.625240
5,1,범죄도시2,이상용,"주식회사 빅펀치픽쳐스,(주)홍필름,(주)비에이엔터테인먼트",NaN,"주식회사 에이비오엔터테인먼트,플러스엠 엔터테인먼트",2022.5.18,개봉영화,장편,한국,...,범죄,15세이상관람가,일반영화,NaN,9.496600,범죄도시2,91.441784,-77.812876,0.807021,67.951857
6,17,브로커,고레에다 히로카즈,영화사 집,NaN,(주)씨제이이엔엠,2022.6.8,개봉영화,장편,한국,...,드라마,12세이상관람가,일반영화,NaN,5.579600,브로커,89.088588,-91.636050,0.384932,88.884182
7,11,비상선언,한재림,"(주)매그넘나인,(주)씨제스엔터테인먼트,씨네주 유한회사",NaN,(주)쇼박스,2022.8.3,개봉영화,장편,한국,...,드라마,12세이상관람가,일반영화,NaN,6.642200,비상선언,89.521035,-89.516013,0.463502,90.304157
8,18,신비한 동물들과 덤블도어의 비밀,데이빗 예이츠,NaN,워너브러더스 코리아(주),워너브러더스 코리아(주),2022.4.13,개봉영화,장편,미국,...,어드벤처,12세이상관람가,일반영화,NaN,6.482335,신비한 동물들과 덤블도어의 비밀,87.551631,-86.945373,0.490578,88.430968
9,15,외계+인 1부,최동훈,(주)케이퍼필름,NaN,(주)씨제이이엔엠,2022.7.20,개봉영화,장편,한국,...,액션,12세이상관람가,일반영화,NaN,7.045200,외계+인 1부,89.920049,-86.982178,0.592070,88.003997


In [61]:
result.to_excel('test.xlsx', index=False)

In [62]:
result.head()

,Unnamed: 0,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,...,장르,등급,영화구분,개봉연도,score,name,pos_avg,neg_avg,pos_ratio,std
0,4,공조2: 인터내셔날,이석훈,"(주)제이케이필름,(주)씨제이이엔엠",NaN,(주)씨제이이엔엠,2022.9.7,개봉영화,장편,한국,...,액션,15세이상관람가,일반영화,NaN,8.791200,공조2_ 인터내셔날,93.297723,-85.397855,0.806760,71.549156
1,5,닥터 스트레인지: 대혼돈의 멀티버스,샘 레이미,NaN,월트디즈니컴퍼니코리아 유한책임회사,월트디즈니컴퍼니코리아 유한책임회사,2022.5.4,개봉영화,장편,미국,...,액션,12세이상관람가,일반영화,NaN,7.185400,닥터 스트레인지_ 대혼돈의 멀티버스,87.393728,-85.986724,0.525724,87.795379
2,20,더 배트맨,맷 리브스,NaN,워너브러더스 코리아(주),워너브러더스 코리아(주),2022.3.1,개봉영화,장편,미국,...,액션,15세이상관람가,일반영화,NaN,7.165800,더 배트맨,87.518002,-87.410354,0.587533,87.248358
3,8,마녀(魔女) Part2. The Other One,박훈정,"(주)영화사 금월,(주)스튜디오앤뉴,(주)페퍼민트앤컴퍼니",NaN,(주)넥스트엔터테인먼트월드(NEW),2022.6.15,개봉영화,장편,한국,...,액션,15세이상관람가,일반영화,NaN,6.843200,마녀(魔女) Part2. The Other One,88.952339,-89.458399,0.512230,90.208853
4,10,미니언즈2,카일 발다,NaN,유니버설픽쳐스인터내셔널 코리아(유),유니버설픽쳐스인터내셔널 코리아(유),2022.7.20,개봉영화,장편,미국,...,애니메이션,전체관람가,일반영화,NaN,7.681496,미니언즈2,91.135864,-85.340623,0.690639,82.625240
